In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Bias function as described by Lennard: https://pubs.acs.org/doi/10.1021/acs.jctc.0c01112

In [ ]:
def dfunct(r_ij, r_kl, r_0, d, k):
    return 0.5 * k * (r_ij + d * r_kl - r_0) ** 2

Evaluate `DFUNCT` at different `R_0`

In [ ]:
def calculate_potential(d, k):
    r_0 = np.linspace(-2, 4, 4)
    r_ij_range = np.linspace(-5, 5, 101)
    r_kl_range = np.linspace(-5, 5, 101)
    r_ij, r_kl = np.meshgrid(r_ij_range, r_kl_range)
    V = np.zeros((4, 101, 101))
    V[0] = dfunct(r_ij, r_kl, r_0[0], d, k)
    V[1] = dfunct(r_ij, r_kl, r_0[1], d, k)
    V[2] = dfunct(r_ij, r_kl, r_0[2], d, k)
    V[3] = dfunct(r_ij, r_kl, r_0[3], d, k)
    return V, r_ij, r_kl

Plot surface maps at different bias distances `R_0`:

In [ ]:
def plot_pes(V, r_ij, r_kl):
    cs = []
    fig, axs = plt.subplots(2, 2, figsize=(14, 14))
    cs.append(axs[0, 0].contour(r_ij, r_kl, V[0]))
    cs.append(axs[0, 1].contour(r_ij, r_kl, V[1]))
    cs.append(axs[1, 0].contour(r_ij, r_kl, V[2]))
    cs.append(axs[1, 1].contour(r_ij, r_kl, V[3]))
  
    for ax, cs_plot in zip(axs.flat, cs):
        ax.set(xlabel='r_ij', ylabel='r_kl')
        ax.clabel(cs_plot, inline=True, fontsize=10)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
    
    plt.show()

### Calculate PES for `d = -1` 

In [ ]:
V, r_ij, r_kl = calculate_potential(-1, 20000)
plot_pes(V, r_ij, r_kl)

### Calculate PES for `d = +1` 

In [ ]:
V, r_ij, r_kl = calculate_potential(1, 20000)
plot_pes(V, r_ij, r_kl)

## Definition of the Potential

In [ ]:
class Potential:
    
    def __init__(self, i, j, k, l, r_0, d, force, verbose=True):
        self.i = i
        self.j = j
        self.k = k
        self.l = l
        self.r_0 = r_0
        self.d = d
        self.force = force
        self.verbose = verbose
    
    def calculate_potential(self):
        r_ij_vec = self.j - self.i
        r_kl_vec = self.k - self.l
        r_ij = np.linalg.norm(r_ij_vec)
        r_kl = np.linalg.norm(r_kl_vec)
        return 0.5 * self.force * (r_ij + self.d * r_kl) ** 2
    
    def calculate_forces(self):
        return self.calculate_force_i_(), self.calculate_force_j_(), self.calculate_force_k_(), self.calculate_force_l_()
    
    def calculate_force_i_(self):
        r_ij_vec = self.j - self.i
        r_kl_vec = self.l - self.k
        r_ij = np.linalg.norm(r_ij_vec)
        r_kl = np.linalg.norm(r_kl_vec)
        prefactor = self.force * (r_ij_vec / r_ij)
        if (self.verbose):
            print(f"prefactor_i: {prefactor}")
            print(f"r_ij_vec: {r_ij_vec}")
            print(f"r_kl_vec: {r_kl_vec}")
            print(f"r_ij: {r_ij}")
            print(f"r_kl: {r_kl}")
        force_i = prefactor * (r_ij + self.d * r_kl - self.r_0)
        return force_i
    
    def calculate_force_j_(self):
        return -self.calculate_force_i_()
    
    def calculate_force_k_(self):
        r_ij_vec = self.j - self.i
        r_kl_vec = self.l - self.k
        r_ij = np.linalg.norm(r_ij_vec)
        r_kl = np.linalg.norm(r_kl_vec)
        prefactor = self.force * self.d * (r_kl_vec / r_kl)
        if (self.verbose):
            print(f"prefactor_i: {prefactor}")
            print(f"r_ij_vec: {r_ij_vec}")
            print(f"r_kl_vec: {r_kl_vec}")
            print(f"r_ij: {r_ij}")
            print(f"r_kl: {r_kl}")
        force_k = prefactor * (r_ij + self.d * r_kl - self.r_0)
        return force_k
    
    def calculate_force_l_(self):
        return -self.calculate_force_k_()

In [ ]:
# example from GROMOS trajectory (step 1)
a = np.array([0.261336286, -0.003121525, -0.040602981])
b = np.array([0.016622976, 0.017632865, -0.0226258])
c = b
d = np.array([-0.211832041, 0.044598137, -0.065052715])
potential1 = Potential(a, b, c, d, 0.0, -1, 20000, verbose=True)
print(f"Forces on atoms: {potential1.calculate_forces()}")
print(f"V_bias: {potential1.calculate_potential()}")

In [ ]:
# example from GROMOS trajectory (step 200)
a = np.array([0.334580982, 0.0461282392, -0.0373055892])
b = np.array([0.0514851911, 0.0311563035, 0.00856868253])
c = b
d = np.array([-0.246007775, 0.0871452496, -0.0550773208])
potential2 = Potential(a, b, c, d, 0.0, -1, 20000, verbose=True)
print(f"Forces on atoms: {potential2.calculate_forces()}")
print(f"V_bias: {potential2.calculate_potential()}")

In [ ]:
# one-dimensional potential
a = np.array([1, 0, 0])
b = np.array([2, 0, 0])
c = b
d = np.array([2.2, 0, 0])
potential3 = Potential(a, b, c, d, 0.0, -1, 20000, verbose=False)
print(f"Forces on atoms: {potential3.calculate_forces()}")
print(f"V_bias: {potential3.calculate_potential()}")

In [ ]:
# one-dimensional potential
a = np.array([1, 0, 0])
b = np.array([2, 0, 0])
c = b
d = np.array([3.2, 0, 0])
potential4 = Potential(a, b, c, d, 0.0, -1, 20000, verbose=False)
print(f"Forces on atoms: {potential4.calculate_forces()}")
print(f"V_bias: {potential4.calculate_potential()}")

In [ ]:
# one-dimensional potential
a = np.array([-1, 0, 0])
b = np.array([2, 0, 0])
c = b
d = np.array([3.0, 0, 0])
potential5 = Potential(a, b, c, d, 0.0, -1, 20000, verbose=False)
print(f"Forces on atoms: {potential5.calculate_forces()}")
print(f"V_bias: {potential5.calculate_potential()}")